In [1]:
# Model-2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import files
uploaded = files.upload()
df2 = pd.read_csv("dataset.csv")
df2.head()

Saving dataset.csv to dataset.csv


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [3]:
# creating a new combined column Timestamp
df2["Timestamp"] = pd.to_datetime(df2["LastUpdatedDate"] + " " + df2["LastUpdatedTime"], format='%d-%m-%Y %H:%M:%S')
df2.sort_values(by=["SystemCodeNumber", "Timestamp"], inplace=True)
df2.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [6]:
# Encode categorical variables
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 1.5, "cycle":0.3}
traffic_levels = {"low": 1.0, "average": 0.5, "high": 0.1}  # lower = worse traffic

df2["VehicleWeight"] = df2["VehicleType"].map(vehicle_weights)
df2["TrafficScore"] = df2["TrafficConditionNearby"].map(traffic_levels)

df2.head(10)


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,VehicleWeight,TrafficScore
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1.0,1.0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1.0,1.0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1.0,1.0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1.0,1.0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,0.5,1.0
5,5,BHMBCCMKT01,577,26.144536,91.736172,177,car,low,3,0,04-10-2016,10:26:00,2016-10-04 10:26:00,1.0,1.0
6,6,BHMBCCMKT01,577,26.144536,91.736172,219,truck,high,6,0,04-10-2016,10:59:00,2016-10-04 10:59:00,1.5,0.1
7,7,BHMBCCMKT01,577,26.144536,91.736172,247,car,average,5,0,04-10-2016,11:25:00,2016-10-04 11:25:00,1.0,0.5
8,8,BHMBCCMKT01,577,26.144536,91.736172,259,cycle,average,5,0,04-10-2016,11:59:00,2016-10-04 11:59:00,0.3,0.5
9,9,BHMBCCMKT01,577,26.144536,91.736172,266,bike,high,8,0,04-10-2016,12:29:00,2016-10-04 12:29:00,0.5,0.1


In [7]:
# Computing Raw Demand with different parameters
α, β, γ, δ, ε = 2.0, 1.0, 1.5, 2.0, 1.0

df2["DemandRaw"] = (
    α * (df2["Occupancy"] / df2["Capacity"])
    + β * df2["QueueLength"].fillna(0)
    - γ * df2["TrafficScore"].fillna(1.0)
    + δ * df2["IsSpecialDay"]
    + ε * df2["VehicleWeight"].fillna(1.0)
)

df2.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,VehicleWeight,TrafficScore,DemandRaw
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1.0,1.0,0.711438
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1.0,1.0,0.721837
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1.0,1.0,1.777296
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1.0,1.0,1.870884
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,0.5,1.0,1.519931


In [9]:
# Normalising demand between 0 and 1
scaler = MinMaxScaler()
df2["DemandNorm"] = scaler.fit_transform(df2[["DemandRaw"]])

# Calculating the final Price
BASE_PRICE = 10
LAMBDA = 0.5
MIN_PRICE = 5
MAX_PRICE = 20

df2["Price"] = BASE_PRICE * (1 + LAMBDA * df2["DemandNorm"])
df2["Price"] = df2["Price"].clip(lower=MIN_PRICE, upper=MAX_PRICE)

# Done! 🔥
print(df2[["SystemCodeNumber", "Timestamp", "DemandRaw", "DemandNorm", "Price"]].head(10))

  SystemCodeNumber           Timestamp  DemandRaw  DemandNorm      Price
0      BHMBCCMKT01 2016-10-04 07:59:00   0.711438    0.081269  10.406343
1      BHMBCCMKT01 2016-10-04 08:25:00   0.721837    0.081774  10.408868
2      BHMBCCMKT01 2016-10-04 08:59:00   1.777296    0.133035  10.665173
3      BHMBCCMKT01 2016-10-04 09:32:00   1.870884    0.137580  10.687900
4      BHMBCCMKT01 2016-10-04 09:59:00   1.519931    0.120535  10.602675
5      BHMBCCMKT01 2016-10-04 10:26:00   3.113518    0.197932  10.989659
6      BHMBCCMKT01 2016-10-04 10:59:00   8.109099    0.440555  12.202774
7      BHMBCCMKT01 2016-10-04 11:25:00   6.106153    0.343277  11.716383
8      BHMBCCMKT01 2016-10-04 11:59:00   5.447747    0.311299  11.556497
9      BHMBCCMKT01 2016-10-04 12:29:00   9.272010    0.497035  12.485173


In [13]:
from pyngrok import ngrok



In [ ]:
# Kill any running tunnels
ngrok.kill()

ngrok.set_auth_token("2zXeD5U43qMqAbY9iIIh57guWE1_4C1NfrnGHuEMnKfdYKfv2")

# Start new tunnel
public_url = ngrok.connect(5006)
print("🌐 Open this Bokeh App URL:", public_url)

# Run Bokeh server in background
!bokeh serve --allow-websocket-origin="*" model2_bokeh.py --port=5006 &

🌐 Open this Bokeh App URL: NgrokTunnel: "https://c10045462933.ngrok-free.app" -> "http://localhost:5006"
2025-07-08 05:51:53,242 Starting Bokeh server version 3.7.3 (running on Tornado 6.4.2)
2025-07-08 05:51:53,243 Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly
2025-07-08 05:51:53,244 User authentication hooks NOT provided (default user enabled)
2025-07-08 05:51:53,247 Bokeh app running at: http://localhost:5006/model2_bokeh
2025-07-08 05:51:53,247 Starting Bokeh server with process id: 7627
2025-07-08 05:52:04,043 WebSocket connection opened
2025-07-08 05:52:04,044 ServerConnection created


In [14]:

bokeh_code = '''
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.plotting import figure, curdoc
from bokeh.driving import linear
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Simulate a preprocessed dataset
n_points = 100
timestamps = pd.date_range(start='2025-07-07 08:00', periods=n_points, freq='30min')
occupancy = np.random.randint(10, 50, size=n_points)
capacity = 50
queue = np.random.randint(0, 10, size=n_points)
traffic = np.random.choice([1.0, 0.5, 0.1], size=n_points)
special_day = np.random.choice([0, 1], size=n_points)
vehicle_weight = np.random.choice([1.0, 0.5, 1.5], size=n_points)

# Demand Function
α, β, γ, δ, ε = 2.0, 1.0, 1.5, 2.0, 1.0
demand_raw = (
    α * (occupancy / capacity)
    + β * queue
    - γ * traffic
    + δ * special_day
    + ε * vehicle_weight
)

scaler = MinMaxScaler()
demand_norm = scaler.fit_transform(demand_raw.reshape(-1, 1)).flatten()

BASE_PRICE = 10
LAMBDA = 0.5
MIN_PRICE = 5
MAX_PRICE = 20
prices = BASE_PRICE * (1 + LAMBDA * demand_norm)
prices = np.clip(prices, MIN_PRICE, MAX_PRICE)

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(title="Real-Time Parking Price Plot (Model 2)",
           x_axis_type='datetime', width=800, height=400)
p.line(x='x', y='y', source=source, line_width=2, color='green')
p.xaxis.formatter = DatetimeTickFormatter(minutes="%H:%M", hours="%H:%M")
p.yaxis.axis_label = "Price ($)"
p.xaxis.axis_label = "Time"

index = dict(i=0)

@linear()
def update(step):
    i = index['i']
    if i >= len(timestamps):
        return
    new_data = dict(x=[timestamps[i]], y=[prices[i]])
    source.stream(new_data, rollover=200)
    index['i'] += 1

curdoc().add_root(column(p))
curdoc().add_periodic_callback(update, 500)
'''

with open("model2_bokeh.py", "w") as f:
    f.write(bokeh_code)